In [2]:
import zipfile
import pandas as pd

import pymorphy3
morph = pymorphy3.MorphAnalyzer()
import random

### Задание 1

In [3]:
pip install pymorphy3

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [4]:
df = pd.read_csv('word_forms.zip', compression='zip', header=0, sep='\t',)

In [5]:
def get_info(word: str):
    return morph.parse(word)[0].tag

get_info('дерево')
     

OpencorporaTag('NOUN,inan,neut sing,nomn')

In [6]:
df['pos'] = df['1gram'].apply(lambda x: get_info(str(x))) # В датасете почему то были float...

In [7]:
def sim_finder(word: str):
    sims = df[df['pos']== get_info(word)]['1gram'].values
    return sims[random.randrange(0, len(sims), 1)]

In [8]:
def sen_maker(sentence: str):
    sen_parsed = sentence.split(' ')
    sen_return = []
    for word in sen_parsed:
        sen_return.append(sim_finder(word))
    return sen_return

In [9]:
sentence = input()
print(sentence)
print(*sen_maker(sentence))


добрый дуб сказал привет и ушел в закат плакать
необъятный оговор настряпал травертин ЧТОБ завертелся Для локоть силиться


### Задание 2

In [ ]:
def lg(x): # находит кол-во необходимых действий  
    n = 0 
    while x != 0: 
        x = x // 3 
        n += 1 
    return n 
 
def solver(l, r): 
    k1 = lg(l) 
    answer = k1 * 2 
    l += 1 
    print(answer)
    while 3 ** k1 <= r: 
        answer += (3 ** (k1) - l) * k1 
        l = 3 ** (k1) 
        k1 += 1 
    print(answer) 
    answer += k1 * (r - l + 1) 
    return answer 
 
 
# это пока не работает а то что сверху работает
def pre_processor(l, r): 
    answer = 0 
    logs3 = [3 ** i for i in range(14)] 
    k = lg(l) 
    answer += k * 2 
    print(answer) 
    for i in range(k+1, 14): 
        if logs3[i] > r: 
            break
        else: 
            answer += (logs3[i] - logs3[i-1])*(i+1) 

    answer += (r - logs3[i-1] + 1) * (i) 
     
    return answer 
     
print(pre_processor(19, 84)) 
print('-------') 
print(solver(19, 84)) 
 
     
print(pre_processor(0,3)) 
print(pre_processor(1,3)) 
print(pre_processor(2, 4)) 
print(pre_processor(1999999, 2000000)) 




### Задание 3

In [ ]:
import re
import inspect
from typing import Callable


find_print_pattern = r"^(?P<indent>\s*)print\((?P<args_and_kwargs>.*?)\)\s*$"
find_func_pattern = r"def.*"

class Logger:

    log_levels = {'debug' : 1, 'info' : 2, 'warning' : 3, 'error' :4 }

    def __init__(self, log_level: str = "warning") -> None:
        self.dec_log_level = self.log_levels[log_level]


    def log(self, log_level: str) -> Callable:
        func_log_level = self.log_levels[log_level]
        modified_code = ""
        
        def wrapper(func: Callable) -> Callable:
            func_code = re.search(find_func_pattern, inspect.getsource(func), re.DOTALL).group(0)
            matches = list(re.finditer(find_print_pattern, func_code, re.MULTILINE))

            for match in matches:
                indents = match.group('indent')
                args_and_kwargs = match.group('args_and_kwargs')
                if func_log_level < self.dec_log_level:
                    repl = f"{indents}print()"
                else:
                    repl = f"{indents}print(f'{log_level.upper()}: ', {args_and_kwargs})"
                func_code = func_code.replace(match.group(0), repl)
            
            exec_globals = func.__globals__.copy()
            exec(func_code, exec_globals)
            modified_func = exec_globals[func.__name__]

            return modified_func
        return wrapper


logger = Logger("info")

@logger.log("warning")
def func1():
    print("Hello from func1")

@logger.log("info")
def func2():
    print("Hello from func2")

@logger.log("debug")
def func3():
    print("Hello from func3")

@logger.log('warning')
def func4():
    print("fdfd")
    print("fdfdf")
func1()
func2()
func3()
func4()
